# ItaData2024
propositional example

In [1]:
using MLJ, Sole
using MLJDecisionTreeInterface
using SoleDecisionTreeInterface
using CategoricalArrays
using DataFrames, JLD2, CSV
using StatsBase, Statistics
using Catch22, Audio911

### Open .jld2 file
the file contains 504 samples of respiratory sound, labeled with 2 classes: healty and pneumonia

In [2]:
ds_path = "/datasets/respiratory_Healthy_Pneumonia"

d = jldopen(string((@__DIR__), ds_path, ".jld2"))
x, y = d["dataframe_validated"]
@assert x isa DataFrame
close(d)

### Audio features extraction function
This function is called for every audio sample and extract 51 features:
26 bands of the mel spectrogram,
13 coefficients of the mfcc
12 spectral features: centroid, crest, entropy, f0, flatness, flux, kurtosis, rolloff, skewness, decrease, slope, spread

In [3]:
nan_replacer!(x::AbstractArray{<:AbstractFloat}) = replace!(x, NaN => 0.0)
nan_replacer!(x::Vector{Vector{Float64}}) = map!(v -> replace!(v, NaN => 0.0), x, x)

function afe(x::AbstractVector{Float64}; get_only_melfreq=false)
    # -------------------------------- parameters -------------------------------- #
    # audio module
    sr = 8000
    norm = true
    speech_detection = false
    # stft module
    stft_length = 256
    win_type = (:hann, :periodic)
    win_length = 256
    overlap_length = 128
    stft_norm = :power                      # :power, :magnitude, :pow2mag
    # mel filterbank module
    nbands = 26
    scale = :mel_htk                        # :mel_htk, :mel_slaney, :erb, :bark
    melfb_norm = :bandwidth                 # :bandwidth, :area, :none
    freq_range = (300, round(Int, sr / 2))
    # mel spectrogram module
    db_scale = false
    # mfcc module
    ncoeffs = 13
    rectification = :log                    # :log, :cubic_root
    dither = true
    # f0 module
    method = :nfc
    f0_range = (50, 400)

    # --------------------------------- functions -------------------------------- #
    # audio module
    audio = load_audio(
        file=x,
        sr=sr,
        norm=norm,
    );

    stftspec = get_stft(
        audio=audio,
        stft_length=stft_length,
        win_type=win_type,
        win_length=win_length,
        overlap_length=overlap_length,
        norm=stft_norm
    );

    # mel filterbank module
    melfb = get_melfb(
        stft=stftspec,
        nbands=nbands,
        scale=scale,
        norm=melfb_norm,
        freq_range=freq_range
    );

    if get_only_melfreq
        return melfb.freq
    end

    # mel spectrogram module
    melspec =  get_melspec(
        stft=stftspec,
        fbank=melfb,
        db_scale=db_scale
    );

    # mfcc module
    mfcc = get_mfcc(
        source=melspec,
        ncoeffs=ncoeffs,
        rectification=rectification,
        dither=dither,
    );

    # f0 module
    f0 = get_f0(
        source=stftspec,
        method=method,
        freq_range=f0_range
    );

    # spectral features module
    spect = get_spectrals(
        source=stftspec,
        freq_range=freq_range
    );

    x_features = hcat(
        melspec.spec',
        mfcc.mfcc',
        f0.f0,
        spect.centroid,
        spect.crest,
        spect.entropy,
        spect.flatness,
        spect.flux,
        spect.kurtosis,
        spect.rolloff,
        spect.skewness,
        spect.decrease,
        spect.slope,
        spect.spread
    );

    nan_replacer!(x_features)

    return x_features
end

afe (generic function with 1 method)

### Compute DataFrame of features

In [4]:
color_code = Dict(:red => 31, :green => 32, :yellow => 33, :blue => 34, :magenta => 35, :cyan => 36)
freq = round.(Int, afe(x[1, :audio]; get_only_melfreq=true))
r_select = r"\e\[\d+m(.*?)\e\[0m"

catch9_f = ["max", "min", "mean", "med", "std", "bsm", "bsd", "qnt", "3ac"]
variable_names = []
for j in catch9_f
    global variable_names = vcat(variable_names...,
        ["\e[$(color_code[:yellow])mmel$i=$(freq[i])Hz->$j\e[0m" for i in 1:26]...,
        ["\e[$(color_code[:red])mmfcc$i->$j\e[0m" for i in 1:13]...,
        "\e[$(color_code[:green])mf0->$j\e[0m", "\e[$(color_code[:cyan])mcntrd->$j\e[0m", "\e[$(color_code[:cyan])mcrest->$j\e[0m",
        "\e[$(color_code[:cyan])mentrp->$j\e[0m", "\e[$(color_code[:cyan])mflatn->$j\e[0m", "\e[$(color_code[:cyan])mflux->$j\e[0m",
        "\e[$(color_code[:cyan])mkurts->$j\e[0m", "\e[$(color_code[:cyan])mrllff->$j\e[0m", "\e[$(color_code[:cyan])mskwns->$j\e[0m",
        "\e[$(color_code[:cyan])mdecrs->$j\e[0m", "\e[$(color_code[:cyan])mslope->$j\e[0m", "\e[$(color_code[:cyan])msprd->$j\e[0m"
    )
end

### Data compression for propositional analysis

In [5]:
catch9 = [
    maximum,
    minimum,
    StatsBase.mean,
    median,
    std,
    Catch22.SB_BinaryStats_mean_longstretch1,
    Catch22.SB_BinaryStats_diff_longstretch0,
    Catch22.SB_MotifThree_quantile_hh,
    Catch22.SB_TransitionMatrix_3ac_sumdiagcov,
]

X = DataFrame([name => Float64[] for name in [match(r_select, v)[1] for v in variable_names]])
audio_feats = [afe(row[:audio]) for row in eachrow(x)]
push!(X, nan_replacer!(vcat([vcat([map(func, eachcol(row)) for func in catch9]...) for row in audio_feats]))...)

yc = CategoricalArray(y);

train_ratio = 0.8

train, test = partition(eachindex(yc), train_ratio, shuffle=true)
# train, test = partition(eachindex(yc), train_ratio, shuffle=false) ### Debug
X_train, y_train = X[train, :], yc[train]
X_test, y_test = X[test, :], yc[test]

println("Training set size: ", size(X_train), " - ", length(y_train))
println("Test set size: ", size(X_test), " - ", length(y_test))

Training set size: (403, 459) - 403
Test set size: (101, 459) - 101


### Train a model

In [6]:
learned_dt_tree = begin
    Tree = MLJ.@load DecisionTreeClassifier pkg=DecisionTree
    model = Tree(max_depth=-1, )
    mach = machine(model, X_train, y_train)
    fit!(mach)
    fitted_params(mach).tree
end

┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /home/paso/.julia/packages/MLJModels/8W54X/src/loading.jl:159


import MLJDecisionTreeInterface ✔


┌ Info: Training machine(DecisionTreeClassifier(max_depth = -1, …), …).
└ @ MLJBase /home/paso/.julia/packages/MLJBase/7nGJF/src/machines.jl:499


mfcc3->min < -0.4241
├─ mfcc2->min < 1.449
│  ├─ mfcc3->med < -1.12
│  │  ├─ Pneumonia (1/1)
│  │  └─ Healthy (81/81)
│  └─ mfcc9->max < 0.7461
│     ├─ mel11=1179Hz->min < 8.438e-6
│     │  ├─ decrs->med < -0.1705
│     │  │  ├─ mfcc11->qnt < 2.1
│     │  │  │  ⋮
│     │  │  │  
│     │  │  └─ mel13=1407Hz->med < 1.292e-5
│     │  │     ⋮
│     │  │     
│     │  └─ mel17=1950Hz->std < 3.321e-5
│     │     ├─ Healthy (12/12)
│     │     └─ Pneumonia (1/1)
│     └─ mel24=3257Hz->qnt < 2.136
│        ├─ Healthy (49/49)
│        └─ mfcc10->3ac < 0.007805
│           ├─ flatn->std < 0.06577
│           │  ⋮
│           │  
│           └─ mfcc3->mean < 0.7573
│              ⋮
│              
└─ sprd->max < 953.4
   ├─ mel25=3491Hz->med < 2.256e-6
   │  ├─ Pneumonia (138/138)
   │  └─ mel23=3037Hz->3ac < 0.01224
   │     ├─ Pneumonia (8/8)
   │     └─ Healthy (2/2)
   └─ cntrd->min < 368.1
      ├─ Pneumonia (3/3)
      └─ Healthy (8/8)


### Model inspection & rule study

In [7]:
sole_dt = solemodel(learned_dt_tree)
# Make test instances flow into the model, so that test metrics can, then, be computed.
apply!(sole_dt, X_test, y_test);
# Print Sole model
printmodel(sole_dt; show_metrics = true, variable_names_map = variable_names);

▣ mfcc3->min < -0.4240806889009767
├✔ mfcc2->min < 1.4486317561728843
│├✔ mfcc3->med < -1.1204737653523038
││├✔ Pneumonia : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
││└✘ Healthy : (ninstances = 23, ncovered = 23, confidence = 0.91, lift = 1.0)
│└✘ mfcc9->max < 0.746071105620268
│ ├✔ mel11=1179Hz->min < 8.438420212132094e-6
│ │├✔ decrs->med < -0.170498600054933
│ ││├✔ mfcc11->qnt < 2.100057267209923
│ │││├✔ mel9=975Hz->3ac < 0.009761300970092177
│ ││││├✔ Healthy : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
│ ││││└✘ Pneumonia : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
│ │││└✘ Pneumonia : (ninstances = 8, ncovered = 8, confidence = 0.88, lift = 1.0)
│ ││└✘ mel13=1407Hz->med < 1.291901452177436e-5
│ ││ ├✔ Healthy : (ninstances = 2, ncovered = 2, confidence = 0.5, lift = 1.0)
│ ││ └✘ Pneumonia : (ninstances = 0, ncovered = 0, confidence = NaN, lift = NaN)
│ │└✘ mel17=1950Hz->std < 3.320789077212966e-5
│ │ ├✔ Healthy : (ninstances = 3

### Extract rules that are at least as good as a random baseline model

In [8]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0);
printmodel.(interesting_rules; show_metrics = true, variable_names_map = variable_names);

▣ (mfcc3->min < -0.4240806889009767) ∧ (mfcc2->min < 1.4486317561728843) ∧ (¬(mfcc3->med < -1.1204737653523038))  ↣  Healthy : (ninstances = 101, ncovered = 23, coverage = 0.23, confidence = 0.91, lift = 1.65, natoms = 3)
▣ (mfcc3->min < -0.4240806889009767) ∧ (¬(mfcc2->min < 1.4486317561728843)) ∧ (mfcc9->max < 0.746071105620268) ∧ (mel11=1179Hz->min < 8.438420212132094e-6) ∧ (decrs->med < -0.170498600054933) ∧ (¬(mfcc11->qnt < 2.100057267209923))  ↣  Pneumonia : (ninstances = 101, ncovered = 8, coverage = 0.08, confidence = 0.88, lift = 1.96, natoms = 6)
▣ (mfcc3->min < -0.4240806889009767) ∧ (¬(mfcc2->min < 1.4486317561728843)) ∧ (mfcc9->max < 0.746071105620268) ∧ (¬(mel11=1179Hz->min < 8.438420212132094e-6)) ∧ (mel17=1950Hz->std < 3.320789077212966e-5)  ↣  Healthy : (ninstances = 101, ncovered = 3, coverage = 0.03, confidence = 1.0, lift = 1.8, natoms = 5)
▣ (mfcc3->min < -0.4240806889009767) ∧ (¬(mfcc2->min < 1.4486317561728843)) ∧ (mfcc9->max < 0.746071105620268) ∧ (¬(mel11=1179H

### Simplify rules while extracting and prettify result

In [9]:
interesting_rules = listrules(sole_dt, min_lift = 1.0, min_ninstances = 0, normalize = true);
printmodel.(interesting_rules; show_metrics = true, syntaxstring_kwargs = (; threshold_digits = 2), variable_names_map = variable_names);

▣ (mfcc3->min < -0.42) ∧ (mfcc2->min < 1.45) ∧ (mfcc3->med ≥ -1.12)  ↣  Healthy : (ninstances = 101, ncovered = 23, coverage = 0.23, confidence = 0.91, lift = 1.65, natoms = 3)
▣ (mfcc3->min < -0.42) ∧ (mfcc2->min ≥ 1.45) ∧ (mfcc9->max < 0.75) ∧ (mel11=1179Hz->min < 0.0) ∧ (decrs->med < -0.17) ∧ (mfcc11->qnt ≥ 2.1)  ↣  Pneumonia : (ninstances = 101, ncovered = 8, coverage = 0.08, confidence = 0.88, lift = 1.96, natoms = 6)
▣ (mfcc3->min < -0.42) ∧ (mfcc2->min ≥ 1.45) ∧ (mfcc9->max < 0.75) ∧ (mel11=1179Hz->min ≥ 0.0) ∧ (mel17=1950Hz->std < 0.0)  ↣  Healthy : (ninstances = 101, ncovered = 3, coverage = 0.03, confidence = 1.0, lift = 1.8, natoms = 5)
▣ (mfcc3->min < -0.42) ∧ (mfcc2->min ≥ 1.45) ∧ (mfcc9->max < 0.75) ∧ (mel11=1179Hz->min ≥ 0.0) ∧ (mel17=1950Hz->std ≥ 0.0)  ↣  Pneumonia : (ninstances = 101, ncovered = 1, coverage = 0.01, confidence = 1.0, lift = 2.24, natoms = 5)
▣ (mfcc3->min < -0.42) ∧ (mfcc2->min ≥ 1.45) ∧ (mfcc9->max ≥ 0.75) ∧ (mel24=3257Hz->qnt < 2.14)  ↣  Healthy : (n

### Directly access rule metrics

In [10]:
readmetrics.(listrules(sole_dt; min_lift=1.0, min_ninstances = 0))

13-element Vector{@NamedTuple{ninstances::Int64, ncovered::Int64, coverage::Float64, confidence::Float64, lift::Float64, natoms::Int64}}:
 (ninstances = 101, ncovered = 23, coverage = 0.22772277227722773, confidence = 0.9130434782608695, lift = 1.6467391304347823, natoms = 3)
 (ninstances = 101, ncovered = 8, coverage = 0.07920792079207921, confidence = 0.875, lift = 1.9638888888888888, natoms = 6)
 (ninstances = 101, ncovered = 3, coverage = 0.0297029702970297, confidence = 1.0, lift = 1.8035714285714284, natoms = 5)
 (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 2.2444444444444445, natoms = 5)
 (ninstances = 101, ncovered = 14, coverage = 0.13861386138613863, confidence = 0.9285714285714286, lift = 1.6747448979591837, natoms = 4)
 (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 1.8035714285714284, natoms = 8)
 (ninstances = 101, ncovered = 2, coverage = 0.019801980198019802, confidence = 1.0, lift

### Show rules with an additional metric (syntax height of the rule's antecedent)

In [11]:
printmodel.(sort(interesting_rules, by = readmetrics); show_metrics = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))), variable_names_map = variable_names);

▣ (mfcc3->min < -0.4240806889009767) ∧ (mfcc2->min ≥ 1.4486317561728843) ∧ (mfcc9->max ≥ 0.746071105620268) ∧ (mel24=3257Hz->qnt ≥ 2.1355321877484625) ∧ (mfcc10->3ac < 0.0078053259871441695) ∧ (flatn->std < 0.06576898424905844) ∧ (mfcc4->mean < -0.027273459477797232) ∧ (kurts->min < 5.8199855433949015)  ↣  Healthy : (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 1.8035714285714284, natoms = 8, height = 7)
▣ (mfcc3->min ≥ -0.4240806889009767) ∧ (sprd->max ≥ 953.4487813406679) ∧ (cntrd->min < 368.1315592426538)  ↣  Pneumonia : (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 2.2444444444444445, natoms = 3, height = 2)
▣ (mfcc3->min ≥ -0.4240806889009767) ∧ (sprd->max < 953.4487813406679) ∧ (mel25=3491Hz->med ≥ 2.256190038408491e-6) ∧ (mel23=3037Hz->3ac < 0.01224364860728497)  ↣  Pneumonia : (ninstances = 101, ncovered = 1, coverage = 0.009900990099009901, confidence = 1.0, lift = 2.2444444444444445, nat

### Pretty table of rules and their metrics

In [12]:
metricstable(interesting_rules; variable_names_map = variable_names, metrics_kwargs = (; round_digits = nothing, additional_metrics = (; height = r->SoleLogics.height(antecedent(r)))))

┌─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────┬────────────┬──────────┬────────────┬────────────┬─────────┬────────┬────────┐
│                                                                                                                                                                                                                                                                                                      Antecedent │ Consequent │ ninstances │ ncovered │   coverage │ confidence │    lift │ natoms │ height │
├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────